In [1]:
from pickle import load
from numpy import array
from numpy import argmax
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu

Using TensorFlow backend.


In [2]:
# load a clean dataset
def load_clean_sentences(filename):
    return load(open(filename, 'rb'))

# fit a tokenizer
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

# max sentence length
def max_length(lines):
    return max(len(line.split()) for line in lines)

# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
    # integer encode sequences
    X = tokenizer.texts_to_sequences(lines)
    # pad sequences with 0 values
    X = pad_sequences(X, maxlen=length, padding='post')
    return X

In [3]:
# map an integer to a word
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

# generate target given source sequence
def predict_sequence(model, tokenizer, source):
    prediction = model.predict(source, verbose=0)[0]
    integers = [argmax(vector) for vector in prediction]
    target = list()
    for i in integers:
        #print(i)
        word = word_for_id(i, tokenizer)
        if word is None:
            break
        target.append(word)
    return ' '.join(target)

# evaluate the skill of the model
def evaluate_model(model, tokenizer, sources, raw_dataset):
    actual, predicted = list(), list()
    for i, source in enumerate(sources):
        # translate encoded source text
        #print("Before ", source.shape)
        source = source.reshape((1, source.shape[0]))
        #print("After ", source.shape)
        translation = predict_sequence(model, eng_tokenizer, source)
        raw_target, raw_src = raw_dataset[i]
        if i < 10:
            print('src=[%s], target=[%s], predicted=[%s]' % (raw_src, raw_target, translation))
        actual.append(raw_target.split())
        predicted.append(translation.split())
    # calculate BLEU score
    print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
    print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
    print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
    print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))


In [8]:
# load datasets

dataset = load_clean_sentences('../data/english-portuguese-training-both.pkl')
train = load_clean_sentences('../data/english-portuguese-training.pkl')
validation = load_clean_sentences('../data/english-portuguese-validation.pkl')

# prepare english tokenizer
eng_tokenizer = create_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])
# prepare german tokenizer
por_tokenizer = create_tokenizer(dataset[:, 1])
por_vocab_size = len(por_tokenizer.word_index) + 1
por_length = max_length(dataset[:, 1])
# prepare data
trainX = encode_sequences(por_tokenizer, por_length, train[:, 1])
validationX = encode_sequences(por_tokenizer, por_length, validation[:, 1])

# load model
model = load_model('model.h5')
# test on some training sequences
print('train')
evaluate_model(model, eng_tokenizer, trainX, train)
# test on some test sequences
#print('test')
#evaluate_model(model, eng_tokenizer, validationX, validation)

train
src=[nao me tente], target=[dont try me], predicted=[dont get me]
src=[eu cheguei cedo], target=[am i early], predicted=[i i right]
src=[quem tem], target=[who has it], predicted=[who it it]
src=[nao se atrase], target=[dont be late], predicted=[dont be]
src=[gosto de voces], target=[i like you], predicted=[i like you]
src=[que pena], target=[what a shame], predicted=[what a pity]
src=[eu estava cansado], target=[i was tired], predicted=[i was sick]
src=[estamos livres], target=[were free], predicted=[were are]
src=[eu estou em paris], target=[im in paris], predicted=[im am paris]
src=[sejam justas], target=[be fair], predicted=[be fair]


C:\ProgramData\Anaconda3\lib\site-packages\nltk\translate\bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\ProgramData\Anaconda3\lib\site-packages\nltk\translate\bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\ProgramData\Anaconda3\lib\site-packages\nltk\translate\bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnin

BLEU-1: 0.075519
BLEU-2: 0.000000
BLEU-3: 0.000000
BLEU-4: 0.000000


### Running the Test data

In [7]:
# load test dataset

test = load_clean_sentences('../data/en_pt_test.pkl')

testX = encode_sequences(por_tokenizer, por_length, array(test)[:, 1])

# load model
model = load_model('model.h5')

print('final test')
evaluate_model(model, eng_tokenizer, array(testX), array(test))

final test
src=[eu perdi meu passaporte], target=[i have lost my passport], predicted=[i lost]
src=[alguem roubou meu dinheiro], target=[someone stole my money], predicted=[is is is]
src=[socorro], target=[help], predicted=[help]
src=[pode trazer a conta], target=[may i have the bill], predicted=[you deep]
src=[eu gostaria de sobremesa], target=[i would like dessert], predicted=[im creative]
src=[eu gostaria de pedir], target=[i would like to order], predicted=[im creative]
src=[posso ver um menu], target=[may i see a menu], predicted=[can i see]
src=[quero uma bebida], target=[i would like a drink], predicted=[i want one]
src=[eu gostaria de um pouco de agua], target=[i would like some water], predicted=[i wont to]
src=[uma mesa para dois], target=[a table for two], predicted=[i cant win]
BLEU-1: 0.097003
BLEU-2: 0.000000
BLEU-3: 0.000000
BLEU-4: 0.000000


C:\ProgramData\Anaconda3\lib\site-packages\nltk\translate\bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\ProgramData\Anaconda3\lib\site-packages\nltk\translate\bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\ProgramData\Anaconda3\lib\site-packages\nltk\translate\bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnin